In [ ]:
import re
import psycopg2
from collections import Counter
from datetime import datetime
from dotenv import load_dotenv
import os

load_dotenv()


In [ ]:

LOG_FILE = "test_access.log"  # <-- Pointing to test log

# ERROR_PATTERN = re.compile(r'(\d{3})\s.*CN=([a-zA-Z0-9.-]+)')
ERROR_PATTERN = re.compile(r'HTTP/1\.\d"\s(\d{3})\s.*CN=([^,\s]+)')

DB_CONFIG = {
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOSTS"),
    "port": 5432,
}

CREATE_TABLE_HTTP = """
CREATE TABLE IF NOT EXISTS http_status_counts (
    id SERIAL PRIMARY KEY,
    status_code INT NOT NULL,
    count INT NOT NULL,
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

CREATE_TABLE_CN = """
CREATE TABLE IF NOT EXISTS cn_counts (
    id SERIAL PRIMARY KEY,
    cn TEXT NOT NULL,
    count INT NOT NULL,
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""
HTTP_STATUS_DESCRIPTIONS = {
    "200": "OK",
    "301": "Moved Permanently",
    "302": "Found",
    "400": "Bad Request",
    "401": "Unauthorized",
    "403": "Forbidden",
    "404": "Not Found",
    "500": "Internal Server Error",
    "502": "Bad Gateway",
    "503": "Service Unavailable",
    "504": "Gateway Timeout"
}
conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()
    
cur.execute(CREATE_TABLE_HTTP)
cur.execute(CREATE_TABLE_CN)


In [ ]:
# Function to parse HTTP statuses from the log file
def parse_http_statuses(log_file):
    status_pattern = re.compile(r'HTTP/1\.\d"\s(\d{3})\s')
    counter = Counter()

    with open(log_file, "r") as file:
        for line in file:
            match = status_pattern.search(line)
            if match:
                status_code = match.group(1)
                counter[status_code] += 1

    return counter

# Function to insert data into the database
def insert_into_db(status_counts):
    conn = None
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()
        
        for status, count in status_counts.items():
            description = HTTP_STATUS_DESCRIPTIONS.get(status, "Unknown")
            cur.execute(
                """
                INSERT INTO http_status_counts (status_code, description, count, timestamp)
                VALUES (%s, %s, %s, %s);
                """,
                (status, description, count, datetime.now()),
            )
        conn.commit()
        cur.close()
        print("✅ HTTP statuses inserted")

    except Exception as e:
        if conn:
            conn.rollback()
        print("❌ Error inserting HTTP statuses:", e)

    finally:
        if conn:
            conn.close()

if __name__ == "__main__":
    LOG_FILE = "test_access.log" 
    http_counts = parse_http_statuses(LOG_FILE)

    if http_counts:
        insert_into_db(http_counts)
    else:
        print(f"⚠️ No HTTP status codes found in the log file.")

In [ ]:
def parse_bad_cns(log_file):
    counter = Counter()

    try:
        with open(log_file, "r") as file:
            for line in file:
                match = ERROR_PATTERN.search(line)
                if match:
                    status_code = match.group(1)
                    cn = match.group(2)
                    if status_code == "400":
                        counter[cn] += 1
        print("CN Counts:", counter)
    except FileNotFoundError:
        print(f"❌ Log file '{log_file}' not found.")
    except Exception as e:
        print(f"❌ Error parsing CNs: {e}")

    return counter

def cn_insert(cn_counts):
    try:
        with psycopg2.connect(**DB_CONFIG) as conn:
            with conn.cursor() as cur:
                for cn, count in cn_counts.items():
                    print(f"Inserting into cn_counts: cn={cn}, count={count}, timestamp={datetime.now()}")
                    cur.execute(
                        "INSERT INTO cn_counts (cn, count, timestamp) VALUES (%s, %s, %s);",
                        (cn, count, datetime.now()),
                    )
            conn.commit()
        print("✅ CN counts inserted")
    except Exception as e:
        print(f"❌ Error inserting CNs: {e}")

if __name__ == "__main__":
    LOG_FILE = "test_access.log" 
    cn_counts = parse_bad_cns(LOG_FILE)

    if cn_counts:
        cn_insert(cn_counts)
    else:
        print(f"⚠️ No HTTP status codes found in the log file.")